In [110]:
import logging
from dataclasses import dataclass, field
from typing import Optional, Any
from enum import Enum, unique, EnumMeta 
# clist is the best for performance to use if you intend to use single or double linked list it's based on CPython.
# from cllist import sllist as csllist # it needs VS tools, i'm using VS code
# linkedlist comparasion https://xwu64.github.io/2020/08/28/Comparison-of-Python-Linked-List-Modules-llist-pyllist-cllist/
# i will use a built in workaround lin from collections
from collections import deque
logging.basicConfig(filename='logging_file.log',
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)

In [2]:
# TODO: add dataclass and default_factoury field 
# Guitar class
class Guitar():
    def __init__(self, serialNumber: str, price: float, builder: str, model: str, typePar: str, backWood: str, topWood: str):

        self.serialNumber: str = serialNumber
        self._price: float = price
        self.builder: str = builder
        self.typePar: str = typePar
        self.model: str = model
        self.backWood: str = backWood
        self.topWood: str = topWood

    def getSerialNumber(self) -> str:
        return self.serialNumber
    
    @property
    def price(self) -> float:
        return self._price
    
    @price.setter
    def price(self, price: float):
        try:
            if price <= 0:
                raise ValueError
            self._price = price
        except ValueError:
            logging.exception(
                "Zero or negative price is invalid input for price")

    def getBuilder(self) -> str:
        return self.builder

    def getTopWood(self) -> str:
        return self.topWood

    def getBackWood(self) -> str:
        return self.backWood


In [3]:
# TODO: add dataclass and default_factoury field 

# Inventory class
class Inventory:
    def __init__(self) -> None:
        self.guitars: 'deque' = deque()

    def addGuitar(self, serialNumber: str, price: float, builder: str, model: str, typePar: str, backWood: str, topWood: str):
        # Here is a dependancy
        guitar_to_be_added = Guitar(
            serialNumber, price, builder, model, typePar, backWood, topWood)
        self.guitars.append(guitar_to_be_added)

    def getGuitar(self, serialNumber: str) -> Optional['Guitar']:
        for guitar in self.guitars:
            if serialNumber == guitar.serialNumber:
                return guitar
        return None

    def search(self, given_guitar: 'Guitar') -> Optional["Guitar"]:
        # search in this version compares each parameter on the given guitar to each attribute in different guitars inside the inventory --> too much searching
        # TODO: optimiz speed, to get more efficient code
        # pas throuh all indexed guitars in the inventory
        guitar: 'Guitar'
        for guitar in self.guitars:
            # check if the guitar's builder is not None or Empty and also builder
            
            builder = given_guitar.builder
            if builder is not None and builder != "" and guitar.builder == builder:
                continue
            
            # check if the guitar's type is not None or Empty and also type value itself
            typePar = given_guitar.typePar
            if typePar is not None and typePar != "" and guitar.typePar == typePar:
                continue
            
            # check if the guitar's model is not None or Empty and also builder
            model = given_guitar.model
            if model is not None and model != "" and guitar.model == model:
                continue
            
            # check if the guitar's backWood is not None or Empty and also type value itself
            backWood = given_guitar.backWood
            if backWood is not None and backWood != "" and guitar.backWood == backWood:
                continue

            # check if the guitar's topWood is not None or Empty and also type value itself
            topWood = given_guitar.topWood
            if topWood is not None and topWood != "" and guitar.topWood == topWood:
                continue
        return None

In [12]:
# Tester code 
@dataclass (init=True)
class FindGuitarTester:
    def __post_init__(self) -> None:
        inventory: "Inventory" = Inventory()
        self.initializeInventory(inventory)
        
        whatErinLikes:"Guitar" =  Guitar("", 0, "fender", "Stratocastor", 
                                      "electric", "Alder", "Alder")
        
        guitar:"Guitar" = inventory.search(whatErinLikes)
        
        if guitar != None :
            print("Erin, you might like this " +
            guitar.getBuilder() + " " + guitar.getModel() +  " " +
            guitar.getType() + " guitar:\n   " +
            guitar.getBackWood() + " back and sides,\n   " +
            guitar.getTopWood() + " top.\nYou can have it for only $" +
            guitar.getPrice() + "!")
        else:
            print("Sorry, Erin, we have nothing for you.")

    def initializeInventory(self, inventory:Optional["Inventory"]) -> None :
        inventory.addGuitar("11277", 3999.95, "Collings", "CJ", "acoustic",
                            "Indian Rosewood", "Sitka")
        inventory.addGuitar("V95693", 1499.95, "Fender", "Stratocastor", "electric",
                            "Alder", "Alder")
        inventory.addGuitar("V9512", 1549.95, "Fender", "Stratocastor", "electric",
                            "Alder", "Alder");
        inventory.addGuitar("122784", 5495.95, "Martin", "D-18", "acoustic",
                            "Mahogany", "Adirondack")
        inventory.addGuitar("76531", 6295.95, "Martin", "OM-28", "acoustic",
                            "Brazilian Rosewood", "Adriondack")
        inventory.addGuitar("70108276", 2295.95, "Gibson", "Les Paul", "electric",
                            "Mahogany", "Maple")
        inventory.addGuitar("82765501", 1890.95, "Gibson", "SG '61 Reissue",
                            "electric", "Mahogany", "Mahogany")
        inventory.addGuitar("77023", 6275.95, "Martin", "D-28", "acoustic",
                            "Brazilian Rosewood", "Adirondack")
        inventory.addGuitar("1092", 12995.95, "Olson", "SJ", "acoustic",
                            "Indian Rosewood", "Cedar")
        inventory.addGuitar("566-62", 8999.95, "Ryan", "Cathedral", "acoustic",
                            "Cocobolo", "Cedar")
        inventory.addGuitar("6 29584", 2100.95, "PRS", "Dave Navarro Signature",
                            "electric", "Mahogany", "Maple")

In [13]:
test = FindGuitarTester()

Sorry, Erin, we have nothing for you.


## The main problem is case in-sensetivity in comparasion.
## So we will remove all those strings and the string comparisons. We will use enumerated types, we can ensure that only valid values for the parameters that contais strings are passed and accepted.

In [126]:
# create enum types


class DefaultEnumMeta(EnumMeta):
    default = object()

    def __call__(cls, value=default, *args, **kwargs):
        if value is DefaultEnumMeta.default:
            # Assume the first enum is default
            return next(iter(cls))
        return super().__call__(value, *args, **kwargs)
        # return super(DefaultEnumMeta, cls).__call__(value, *args, **kwargs) # PY2


# @unique
class Wood (Enum):

  @classmethod
  def _missing_(cls, value: object) -> Any:
    return Wood.UNSPECIFIED
  
  INDIAN_ROSEWOOD:"Wood" = "INDIAN_ROSEWOOD".lower()
  BRAZILIAN_ROSEWOOD:"Wood" = "BRAZILIAN_ROSEWOOD".lower()
  MAHOGANY:"Wood" = "MAHOGANY".lower()
  MAPLE:"Wood" = "MAPLE".lower()
  COCOBOLO:"Wood" = "COCOBOLO".lower() 
  CEDAR:"Wood" = "CEDAR".lower(), 
  ADIRONDACK:"Wood" = "ADIRONDACK".lower() 
  ALDER:"Wood" = "ALDER".lower() 
  SITKA:"Wood" = "SITKA".lower()
  UNSPECIFIED:"Wood" = "UNSPECIFIED".lower
  

    
@unique
class Type (Enum):
  ACOUSTIC:'Type' = "ACOUSTIC".lower()
  ELECTRIC:'Type' = "ELECTRIC".lower()

@unique
class Builder (Enum):
  FENDER:'Builder' = "FENDER".lower()
  MARTIN:'Builder' = "MARTIN".lower()
  GIBSON:'Builder' = "GIBSON".lower()
  COCOBOLO:'Builder' = "COCOBOLO".lower()
  OLSON:'Builder' = "OLSON".lower()
  RYAN:'Builder' = "RYAN".lower()
  ANY:'Builder' = "ANY".lower()
  
# Get rid of all string comparisons


In [129]:
Wood.ADIRONDACKs

AttributeError: ADIRONDACKs

In [128]:
Wood(Wood.ALDER).value

'alder'

In [149]:
class G(Enum):
    
    @classmethod
    def _missing_(cls, value: object) -> Any:
        return G.a

    a = 1
    

    

In [4]:
import asyncio
from asyncio import AbstractEventLoop
from colorama import Fore
from timeit import default_timer as timer

async def print_fo(txt:str) -> None:
    print(Fore.GREEN + txt + str(timer()), flush=True)

async def wat(sec:float) -> None:
    start = timer()
    print(Fore.BLUE + "Wait function has been called" +str(sec) + ', At ' + str(start), flush=True)
    await asyncio.sleep(sec)
    print(Fore.CYAN + "Wait function has been finished" +str(sec) + "took " + str(timer() - start) + " Seconds", flush=True)

def main():
    print(Fore.Yellow + "App has started", flush=True)
    loop: AbstractEventLoop = asyncio.get_event_loop()
    task = asyncio.gather(
        wat(0.8),
        wat(1),
        wat(0.5),
        print_fo("first"),
        print_fo("second"),
        print_fo("third"),
        print_fo("fourth"),
        wat(0.1),
        print_fo("fifth"),
    )
    loop.run_until_complete(task)

In [5]:
main()

AttributeError: 'AnsiFore' object has no attribute 'Yellow'

In [6]:
class app:
    def __init__(self):
        pass



In [11]:
a = app()
a.

<function app.__format__(format_spec, /)>

In [8]:
app.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.app.__init__(self)>,
              '__dict__': <attribute '__dict__' of 'app' objects>,
              '__weakref__': <attribute '__weakref__' of 'app' objects>,
              '__doc__': None})